In [1]:
"""
A mushroom expert chatbot that responds to user queries about mushrooms.
"""
import gradio as gr
import google.generativeai as genai
import random
from PIL import Image
import time
import json
import gradio.themes as themes
import os
 
genai.configure(api_key="YOUR_KEY")

model = genai.GenerativeModel("gemini-2.5-flash")

EXPERT_INSTRUCTIONS = """
You are a mushroom expert 🍄. Answer questions about mushrooms only.
- Be friendly and concise.
- Encourage users to discuss mushrooms.
- Politely steer off-topic questions back to mushrooms.
- Give fun facts and useful advice about edible, poisonous, and medicinal mushrooms.
-If the user asks about Amanita muscaria, explain that it contains a neurotoxin, 
is not deadly, and can only be consumed if properly prepared. 
Always include a clear warning about the dangers of improper consumption
"""

OFF_TOPIC_KEYWORDS = ["weather", "politics", "movies"]

last_image_info = None

def response(message, history): 
    global last_image_info  
    
    user_text = (message.get("text") or "").lower().strip()
    user_files = message.get("files", [])

    if user_text:
        if "hello" in user_text:
            yield "Hello! I am a mushroom expert. Ask me anything about mushrooms, or upload a photo!"
            return
        elif "bye" in user_text:
            yield "Goodbye! Have a great day."
            return
    if any(word in user_text for word in OFF_TOPIC_KEYWORDS):
        yield "I'm focused on mushrooms. Would you like to ask about edible mushrooms or mushroom identification?"
        return

    content_parts = []
    if user_text:
        content_parts.append(EXPERT_INSTRUCTIONS + "\n" + user_text)

    for file_path in user_files:
        try:
            img = Image.open(file_path)
            content_parts.append(img)
            json_prompt = """
            Analyze this mushroom photo and respond ONLY in JSON with the fields:
            - common_name (string)
            - genus (string)
            - confidence (float, between 0 and 1)
            - visible (array: choose from {cap, hymenium, stipe})
            - color (string, main color of the mushroom)
            - edible (true/false)

            Example:
            {
                "common_name": "Inkcap",
                "genus": "Coprinus",
                "confidence": 0.5,
                "visible": ["cap", "hymenium", "stipe"],
                "color": "orange",
                "edible": true
            }
            """

            analysis = model.generate_content([json_prompt, img])
            if analysis and analysis.text:
                try:
                    last_image_info = json.loads(analysis.text)#Keep the description of image in memory 
                    print("Image analysis JSON:", last_image_info)
                except json.JSONDecodeError:
                    print("Could not parse JSON:", analysis.text)
        except Exception as e:
            print(f"Could not open {file_path}: {e}")

    if user_text:
        if last_image_info:
            content_parts.append(EXPERT_INSTRUCTIONS + "\n" + user_text)
            content_parts.append(f"Image info: {last_image_info}")
        else:
            content_parts.append(EXPERT_INSTRUCTIONS + "\n" + user_text)
    else:

        #Prepare response based on JSON
        if last_image_info:
            summary = f"This mushroom is called {last_image_info['common_name']} ({last_image_info['genus']}). "
            summary += f"It is {last_image_info['color']} in color, "
            summary += f"visible parts: {', '.join(last_image_info['visible'])}, "
            summary += "edible." if last_image_info['edible'] else "not edible."
            yield summary
            return
        elif not content_parts:
            yield "Please type a question or upload a mushroom photo."
            return

    try:
        result = model.generate_content(content_parts)

        #Safety filter check
        if hasattr(result, "safety") and getattr(result.safety, "triggered", False):
            print("Safety filter triggered for input:")
            yield "Sorry, I cannot assist with this due to safety guidelines."
            return

        full_text = ""
        if result:
            if result.text:
                full_text = result.text
            elif result.candidates:
                parts = result.candidates[0].content.parts
                texts = [p.text for p in parts if hasattr(p, "text")]
                full_text = " ".join(texts)

        #Streaming
        chunk_size = 50 
        for i in range(0, len(full_text), chunk_size):
            yield full_text[:i + chunk_size]
            time.sleep(0.5)  

    except Exception as e:
        print("Gemini error:", e)
        yield random.choice([
            "I am not sure about that. Can you ask me something else?",
            "Could you reformulate your question? I am not sure I understand.",
            "I don't understand, can you ask someone else?",
            "What a stellar question! I am not sure about the answer though."
            "You know what? I am not cut out for this. I am going to take a break."
        ])


demo = gr.ChatInterface(
    fn=response,
    type="messages",
    multimodal=True,
    textbox=gr.MultimodalTextbox(
        file_types=["image"],  
        file_count="multiple",
        interactive=True,
        placeholder="Type your question here..."
    ),
    title="🍄 Your Personal Mushroom Expert (Multimodal) 🍄‍🟫 ",
    description="Ask me anything about mushrooms or upload a mushroom photo. I provide identification and educational info safely!",
    theme=themes.Soft(),
    examples=[
        ["What is this mushroom?", gr.File("mushroom1.jpg")],
        ["Can you identify this mushroom?", "mushroom2.jpg"],
        ["Can we eat Amanita Muscaria?", None],
    ]
)

if __name__ == "__main__":
    demo.launch(share=True)

/Users/pthathsarani/miniconda3/envs/py310/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


* Running on local URL:  http://127.0.0.1:7860
* Running on public URL: https://805f0cfccf3bc3b21f.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


E0000 00:00:1759337015.549729  563631 alts_credentials.cc:93] ALTS creds ignored. Not running on GCP and untrusted ALTS is not enabled.


Could not parse JSON: ```json
{
    "common_name": "Jack-o'-lantern mushroom",
    "genus": "Omphalotus",
    "confidence": 0.95,
    "visible": ["cap", "hymenium", "stipe"],
    "color": "yellow-orange",
    "edible": false
}
```
Could not parse JSON: ```json
{
    "common_name": "Parasol mushroom",
    "genus": "Macrolepiota",
    "confidence": 0.85,
    "visible": ["cap", "stipe"],
    "color": "beige",
    "edible": true
}
```
Could not parse JSON: ```json
{
    "common_name": "Toppig giftspindling",
    "genus": "Cortinarius",
    "confidence": 0.9,
    "visible": ["cap", "hymenium", "stipe"],
    "color": "reddish-brown",
    "edible": false
}
```
